In [2]:
#Get Training Files
import os
import pandas as pd
in_data_array={}
out_data_array={}
for file in os.listdir('/kaggle/input/nfl-big-data-bowl-2026-prediction/train'):
    if file[0:2]=='in':
        week=int(file[-6:-4])
        dataFrame=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+file)
        in_data_array[week]=dataFrame
    elif file[0:2]=='ou':
        week=int(file[-6:-4])
        dataFrame=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+file)
        out_data_array[week]=dataFrame
print("done")

done


Will Seperate Training Data into Training and testing data to predict player position. For polynomial regression it is important to note that there are 8 gaurenteed frames per frame.

In [5]:
weeks=in_data_array.keys()
weeks=list(weeks)
weeks.sort()
print(weeks)
feature_names=[]
across_years_frame=pd.DataFrame(columns=feature_names)
min_num_players=-1
min_num_frames=-1
week=0
w=0
for i in weeks:
    map_plays_to_players={}
    for j in range(len(in_data_array[i])):
        game_play_key=str(in_data_array[i].loc[j,'game_id'])+' '+str(in_data_array[i].loc[j,'play_id'])
        if not game_play_key in map_plays_to_players:
            player=in_data_array[i].loc[j,'nfl_id']
            map_in_map={"Play Count":1,"Players":[player]}
            map_plays_to_players[game_play_key]=map_in_map
        else:
            player=in_data_array[i].loc[j,'nfl_id']
            map_plays_to_players[game_play_key]['Play Count']+=1
            if not player in map_plays_to_players[game_play_key]["Players"]:
                map_plays_to_players[game_play_key]["Players"].append(player)
    for key in map_plays_to_players.keys():
        if min_num_players==-1:
            min_num_players=len(map_plays_to_players[key]["Players"])
            min_num_frames=map_plays_to_players[key]["Play Count"]
        else:
            curr_play=len(map_plays_to_players[key]["Players"])
            curr_frames=map_plays_to_players[key]["Play Count"]
            if curr_play<min_num_players:
                week=key
                w=i
                min_num_players=curr_play
            if curr_frames<min_num_frames:
                min_num_frames=curr_frames

print(min_num_players,min_num_frames)
print(week,w)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
1 35
2023112606 4180 12


In [10]:
#Acquire Actual Data
actual_data=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv')
print(actual_data)

         game_id  play_id  nfl_id  frame_id
0     2024120805       74   54586         1
1     2024120805       74   54586         2
2     2024120805       74   54586         3
3     2024120805       74   54586         4
4     2024120805       74   54586         5
...          ...      ...     ...       ...
5832  2025010515     3902   55112        26
5833  2025010515     3902   55112        27
5834  2025010515     3902   55112        28
5835  2025010515     3902   55112        29
5836  2025010515     3902   55112        30

[5837 rows x 4 columns]


In [ ]:
#Write to output file
with open('/kaggle/working/submission.csv','w+') as file:
    file.write("id,x,y")
